In [25]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [22]:
%run -i ../../zkstats/core.py

In [23]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Verifier calls `export_onnx` with dummy data, to generate the onnx model. Dummy data is used to infer the shape of the model

### Output
- `verifier.onnx` file

In [24]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer. Just need to be the same len as private data
dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.mean(dummy_data_tensor)




In [25]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

!@# circuit.eval= verifier_model()


## Step 2
- Prover gets the model from the verifier.
- Prover overrides the `dummy_theory_output` with `theory_output`
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`

In [26]:
# prover calculates settings, send to verifier

theory_output = torch.mean(data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [5], "resources", settings_path)

Theory_output:  tensor(49.9700)
!@# data= [33. 75. 38. 38. 70. 44. 34. 67. 54. 78. 80. 21. 41. 47. 57. 50. 65. 43.
 51. 54. 62. 68. 45. 39. 51. 48. 48. 42. 37. 75. 40. 48. 65. 26. 42. 53.
 51. 56. 74. 54. 55. 15. 58. 46. 64. 59. 39. 36. 62. 39. 72. 32. 82. 76.
 88. 51. 44. 35. 18. 53. 52. 45. 64. 31. 32. 61. 66. 59. 50. 69. 44. 22.
 45. 45. 46. 42. 83. 53. 53. 69. 53. 33. 48. 49. 34. 66. 29. 66. 52. 45.
 83. 54. 53. 31. 71. 60. 30. 33. 43. 26. 55. 56. 56. 54. 57. 68. 58. 61.
 62. 38. 52. 74. 76. 37. 42. 54. 38. 38. 30. 31. 52. 41. 69. 40. 46. 69.
 29. 28. 66. 41. 40. 36. 52. 58. 46. 42. 85. 45. 70. 49. 48. 34. 18. 39.
 64. 46. 54. 42. 45. 64. 46. 68. 46. 54. 47. 41. 69. 27. 61. 37. 25. 66.
 30. 59. 67. 34. 36. 40. 55. 58. 74. 55. 66. 55. 72. 40. 27. 38. 74. 52.
 45. 40. 35. 46. 64. 41. 50. 45. 42. 22. 25. 55. 39. 58. 56. 62. 55. 65.
 57. 34. 44. 47. 70. 60. 34. 50. 43. 60. 66. 46. 58. 76. 40. 49. 64. 45.
 22. 50. 34. 44. 76. 63. 59. 36. 59. 47. 70. 64. 44. 55. 50. 48. 66. 40.
 76. 48. 

## Step 3
- Prover generates proof with `prover_gen_proof`:
    - 1. Compile circuit with `compile_circuit`
        - Inputs: onnx model, settings path
        - Outputs: compiled 
    - 2. Generate witness with
- Verifier generates keys with 

In [27]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

==== setting up ezkl ====

spawning module 0
spawning module 2


spawning module 0
spawning module 2


Time setup: 1.709338903427124 seconds
Theory output:  tensor(49.9700)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.96875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[17236403609878300513, 8975289109583201277, 11033320042856086930, 2298871365537728986], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [4072474832971751158, 13621019950687641747, 8349885009801153069, 3312433242047143324]]], 'proof': '27e50c37ff33c91093ef0778de450099902dbf8d6bce93390ddf690f57e759e4144253e4aa3ac7ed6ec0abc744ddf5c7c947bfbcb74c2b1592c7857be0fb929b2ac29538ab545f9152089ed7f166a7be795c84f3f44fa4d8d3935bec343228460d184de145c5a7e4b31e6f7badc50ef58508ce9e3a8fbaf7e9eacb7c2e1d0c7b033c0e9872052460b2864eafd7ad7e97579122bc880d1bd4befbd3056cfbe7bd20c6794ed38defc99b8bd84a44fc763c92f88d05158ea618629706cebd069ea8289ae1df8d2ad42c06d95320159cff8c67a4be754ae6d58647dadf2bafbe2ffd0848965ed043ebe9674512b15666f228e568fc1f43c1cd58bff898ac7e266bf6124fbe5f62fcb31b8f150047735324f98f0df642ffffba29cf98e413df581ebf285b762135fcda74403913881f772df814bce99665dbde53c9edcb92e2b2b50408a4a10eb7ec7c8f1908a93c8cccf364fadfd7b720e03bf6d1f5e43f5c67bc

In [28]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[17236403609878300513, 8975289109583201277, 11033320042856086930, 2298871365537728986], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [4072474832971751158, 13621019950687641747, 8349885009801153069, 3312433242047143324]]]
proof boolean:  1.0
proof result 1 : 49.96875
verified
